# Gas Network Problem

There is a network of nodes where some gas is either produced or consumed. The goal of the problem is to find the best way to transport the gas from the production nodes to the consumption nodes.

You have to decide whether a pipeline is constructed between two nodes or not. A pipeline has a capacity and a cost to be opened (i.e, constructed). Once the pipeline is opened, you also have a cost for each unit of gas that is transported through this pipeline. Finally, there is a penalty for each unit of produced gas which is not consumed.

#### You want to minimize the total cost.

In [4]:
# consumption (demand)
d = [ 0, 50, 95, 10, 73, 55, 125, 32, 40, 20 ]
# production (maximum generation)
gmax = [ 500, 0, 0, 500, 0, 0, 500, 0, 0, 0 ]
#gmax = [ 500, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
N = len(d)
NODES = range(N)
print('generation - demand? (%d): %d' % (N, sum(gmax) - sum(d)))

generation - demand? (10): 1000


In [5]:
# capacity of the arcs
ca = [ [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
       [20, 30, 40, 50, 60, 70, 80, 90, 100, 10],
       [30, 40, 50, 60, 70, 80, 90, 100, 10, 20],
       [40, 50, 60, 70, 80, 90, 100, 10, 20, 30],
       [50, 60, 70, 80, 90, 100, 10, 20, 30, 40],
       [60, 70, 80, 90, 100, 10, 20, 30, 40, 50],
       [70, 80, 90, 100, 10, 20, 30, 40, 50, 60],
       [80, 90, 100, 10, 20, 30, 40, 50, 60, 70],
       [90, 100, 10, 20, 30, 40, 50, 60, 70, 80],
       [100, 10, 20, 30, 40, 50, 60, 70, 80, 90]
     ]

# linear variable cost: the cost of transporting one unit of gas
vc = 1

# unsatisfied demand: penalty for each unit of gas which is not consumed or produced
p = 100

# linear fixed cost: the cost of opening an arc is proportional to its capacity
fc = [ [10 * c for c in row] for row in ca]



### Exercices
#### 1. Find the formulation of the problem

Here is the formulation

|           |              |
| ----------|:-------------|
| $d_i$  |  gas demand at node $i$  |
| $gmax_i$ | maximum gas production at node $i$ |
| $vc$  | cost of transporting one unit of gas through a pipeline  |
| $fc_{ij}$  | cost of opening pipeline $(i,j)$  |
| $ca_{i,j}$  |  capacity of pipeline $(i,j)$  |
| $p$  | penalty for each unit of gas which is not consumed or produced  |
| $g_i$ | gas produced at node $i$ |
| $x_{i,j}$  | volume of gas going through pipeline $(i,j)$  |
| $y_{ij}$  | bynary decision, =1 if pipeline $(i,j)$ is open, 0 otherwise  |
| $z_i$  | demand which is not fulfilled at node $i$ |

\begin{align*}
\min & \sum_{i,j} vc*x_{i,j} + \sum_{i,j} fc_{i,j}*y_{i,j} + \sum_i p*z_i & \\
\text{subject to:} && \\
& \sum_j x_{j,i} + g_i = \sum_j x_{j,i} + d_i - z_i & \forall i \\
& x_{i,j} \leq ca_{i,j} * y_{i,j} & \forall i,j \\
& g_i \leq gmax_i & \forall i \\
& g_i, x_{i,j}, z_i \geq 0 & \forall i,j 
& y_{i,j} \in \mathbb{B}
\end{align*}

#### 2. Implement the model

In [6]:
from docplex.mp.model import Model
mdl = Model(name='gas network')

x = mdl.continuous_var_matrix(N,N,lb=0,name='x')
g = mdl.continuous_var_list(N,lb=0,ub=gmax,name='g')
z = mdl.continuous_var_list(N,lb=0,name='z')
y = mdl.binary_var_matrix(N,N,name='y')

for i in NODES:
    for j in NODES:
        mdl.add_constraint(x[i,j] <= ca[i][j] * y[i,j])
for i in NODES:
    mdl.add_constraint(sum(x[j,i] for j in NODES) + g[i] == sum(x[i,j] for j in NODES) + d[i] - z[i],'flow_%d' % i)

mdl.minimize(sum(vc * x[i,j] for i in NODES for j in NODES) + 
             sum(fc[i][j] * y[i,j] for i in NODES for j in NODES) +
             sum(p * z[i] for i in NODES) + sum(p * (gmax[i] - g[i]) for i in NODES))
msol = mdl.solve()
print(msol)


solution for: gas network
objective: 104470
x_0_2=30.000
x_0_5=55.000
x_3_1=50.000
x_3_2=60.000
x_3_4=73.000
x_3_9=20.000
x_6_7=32.000
x_6_8=45.000
x_8_2=5.000
g_0=85.000
g_3=213.000
g_6=202.000
y_0_2=1
y_0_5=1
y_3_1=1
y_3_2=1
y_3_4=1
y_3_9=1
y_6_7=1
y_6_8=1
y_8_2=1



In [7]:
print('Cost: %.2f' % mdl.objective_value)
for i in NODES:
    if z[i].solution_value > 0:
        print('Demand not met for %d: %.2f' &(i, z[i].solution_value))
for i in NODES:
    for j in NODES:
        if x[i,j].solution_value > 0:
            print('Flow from %d to %d: %.2f / %d' %(i, j, x[i,j].solution_value, ca[i][j]))

Cost: 104470.00
Flow from 0 to 2: 30.00 / 30
Flow from 0 to 5: 55.00 / 60
Flow from 3 to 1: 50.00 / 50
Flow from 3 to 2: 60.00 / 60
Flow from 3 to 4: 73.00 / 80
Flow from 3 to 9: 20.00 / 30
Flow from 6 to 7: 32.00 / 40
Flow from 6 to 8: 45.00 / 50
Flow from 8 to 2: 5.00 / 10
